# Tensorflow

In [1]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,
)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

2025-04-10 11:14:54.504786: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 11:14:56.060533: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2025-04-10 11:14:56.060609: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2025-04-10 11:14:56.065346: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
Dropped Escape call with ulEscapeCode : 0x03007703
Dropped Escape call with ulEscapeCode : 0x03007703
2025-04-10 11:14:56.620228: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.
2025-04-10 11:14:57.306358: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/5


2025-04-10 11:15:01.961283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  3/782 [..............................] - ETA: 29s - loss: 6.9640 - accuracy: 0.0156  

2025-04-10 11:15:02.834580: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-10 11:15:02.834639: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 13891 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2025-04-10 11:15:02.840065: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-10 11:15:02.840119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 13891 MB memory) -> physical PluggableDevice (device: 0, name: 

782/782 [==============================] - 32s 36ms/step - loss: 4.5455 - accuracy: 0.0726
Epoch 2/5
782/782 [==============================] - 28s 36ms/step - loss: 4.4741 - accuracy: 0.0806
Epoch 3/5
782/782 [==============================] - 28s 36ms/step - loss: 4.0176 - accuracy: 0.1121
Epoch 4/5
782/782 [==============================] - 28s 36ms/step - loss: 3.8687 - accuracy: 0.1445
Epoch 5/5
782/782 [==============================] - 28s 36ms/step - loss: 3.6197 - accuracy: 0.1829


# Pytorch

In [3]:

import torch
import torch_directml

dml = torch_directml.device()
print(dml)

tensor1 = torch.tensor([1]).to(dml)
tensor2 = torch.tensor([2]).to(dml)

dml_algebra = tensor1 + tensor2
print(dml_algebra.item())

privateuseone:0
3


# Pytorch Performance Comparison

In [4]:
import torch
import torch_directml
import time

# Function to perform a complex operation
def complex_operations(tensor):
    # Perform a series of intensive operations
    result_mm = torch.mm(tensor, tensor)
    result_elementwise = tensor * tensor
    result_sum = torch.sum(result_elementwise)
    return result_mm, result_elementwise, result_sum

# Check if DirectML is available
dml = torch_directml.device()
device = dml if dml else 'cpu'

# Define a very large size for the matrices
N = 10000  # Adjust this size based on your system's capabilities

# Create random tensors
tensor1 = torch.rand(N, N, device=device)
tensor2 = torch.rand(N, N, device=device)

# Time the DirectML operation if available
if device != 'cpu':
    start_time = time.time()
    
    # Perform complex operations on DirectML
    result_mm_dml, result_elementwise_dml, result_sum_dml = complex_operations(tensor1)
    
    dml_time = time.time() - start_time
else:
    dml_time = None

# Move tensors to CPU for CPU operation
tensor1_cpu = tensor1.cpu()
tensor2_cpu = tensor2.cpu()

# Time the CPU operation
start_time = time.time()

# Perform the same complex operations on CPU
result_mm_cpu, result_elementwise_cpu, result_sum_cpu = complex_operations(tensor1_cpu)

cpu_time = time.time() - start_time

# Print the results
if dml_time is not None:
    print(f"Time taken on DirectML (GPU): {dml_time:.4f} seconds")
    print(f"DirectML Result Sum: {result_sum_dml.item():.4f}")
    # To avoid overwhelming the output, only show the shape of the results
    print(f"DirectML Result MM Shape: {result_mm_dml.shape}")
    print(f"DirectML Result Elementwise Shape: {result_elementwise_dml.shape}")

print(f"Time taken on CPU: {cpu_time:.4f} seconds")
print(f"CPU Result Sum: {result_sum_cpu.item():.4f}")
# To avoid overwhelming the output, only show the shape of the results
print(f"CPU Result MM Shape: {result_mm_cpu.shape}")
print(f"CPU Result Elementwise Shape: {result_elementwise_cpu.shape}")

Time taken on DirectML (GPU): 0.2036 seconds
DirectML Result Sum: 33337510.0000
DirectML Result MM Shape: torch.Size([10000, 10000])
DirectML Result Elementwise Shape: torch.Size([10000, 10000])
Time taken on CPU: 4.7501 seconds
CPU Result Sum: 33337508.0000
CPU Result MM Shape: torch.Size([10000, 10000])
CPU Result Elementwise Shape: torch.Size([10000, 10000])
